In [ ]:
# default_exp constants


In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# BigEarthNet Constants
> A collection of all of the common BEN constants that are frequently looked up.

In [ ]:
# export
import fastcore.all as fc
import natsort
import rich


In [ ]:
# export


def _generate_old2new_labels_dict():
    """
    Code used to generate Old2New_labels dictionary.
    Only run once and saved the output as a constant.
    """
    import requests

    src_url = "https://git.tu-berlin.de/rsim/BigEarthNet-S2_19-classes_models/-/raw/master/label_indices.json"
    r = requests.get(src_url)
    src = r.json()
    original_labels = list(src["original_labels"].keys())
    assert len(original_labels) == 43
    new_labels = list(src["BigEarthNet-19_labels"].keys())
    assert len(new_labels) == 19
    label_conv = src["label_conversion"]
    assert len(label_conv) == len(new_labels)

    # d = {}
    # for new_label_idx, old_label_idxs in enumerate(label_conv):
    #     for old_label_idx in old_label_idxs:
    #         d[original_labels[old_label_idx]] = new_labels[new_label_idx]

    # print(d)

    old2new_label_mapping = {
        original_labels[old_idx]: new_labels[new_label_idx]
        for new_label_idx, row in enumerate(src["label_conversion"])
        for old_idx in row
    }

    deleted_labels = set(original_labels) - set(old2new_label_mapping.keys())

    for l in deleted_labels:
        old2new_label_mapping[l] = None

    assert len(old2new_label_mapping.keys()) == len(original_labels)

    return old2new_label_mapping


In [ ]:
# hide
# only run manually, if something has changed or if the site has moved again
# _generate_old2new_labels_dict()


In [ ]:
# export
OLD2NEW_LABELS_DICT = {
    "Continuous urban fabric": "Urban fabric",
    "Discontinuous urban fabric": "Urban fabric",
    "Industrial or commercial units": "Industrial or commercial units",
    "Non-irrigated arable land": "Arable land",
    "Permanently irrigated land": "Arable land",
    "Rice fields": "Arable land",
    "Vineyards": "Permanent crops",
    "Fruit trees and berry plantations": "Permanent crops",
    "Olive groves": "Permanent crops",
    "Annual crops associated with permanent crops": "Permanent crops",
    "Pastures": "Pastures",
    "Complex cultivation patterns": "Complex cultivation patterns",
    "Land principally occupied by agriculture, with significant areas of natural vegetation": "Land principally occupied by agriculture, with significant areas of natural vegetation",
    "Agro-forestry areas": "Agro-forestry areas",
    "Broad-leaved forest": "Broad-leaved forest",
    "Coniferous forest": "Coniferous forest",
    "Mixed forest": "Mixed forest",
    "Natural grassland": "Natural grassland and sparsely vegetated areas",
    "Sparsely vegetated areas": "Natural grassland and sparsely vegetated areas",
    "Moors and heathland": "Moors, heathland and sclerophyllous vegetation",
    "Sclerophyllous vegetation": "Moors, heathland and sclerophyllous vegetation",
    "Transitional woodland/shrub": "Transitional woodland, shrub",
    "Beaches, dunes, sands": "Beaches, dunes, sands",
    "Inland marshes": "Inland wetlands",
    "Peatbogs": "Inland wetlands",
    "Salt marshes": "Coastal wetlands",
    "Salines": "Coastal wetlands",
    "Water courses": "Inland waters",
    "Water bodies": "Inland waters",
    "Coastal lagoons": "Marine waters",
    "Estuaries": "Marine waters",
    "Sea and ocean": "Marine waters",
    "Airports": None,
    "Bare rock": None,
    "Dump sites": None,
    "Port areas": None,
    "Road and rail networks and associated land": None,
    "Mineral extraction sites": None,
    "Construction sites": None,
    "Sport and leisure facilities": None,
    "Burnt areas": None,
    "Intertidal flats": None,
    "Green urban areas": None,
}

OLD_LABELS = sorted(tuple({k for k in OLD2NEW_LABELS_DICT.keys()}))
NEW_LABELS = sorted(tuple({v for v in OLD2NEW_LABELS_DICT.values() if v is not None}))


In [ ]:
# hide
assert len(NEW_LABELS) == 19
assert len(OLD_LABELS) == 43


In [ ]:
# export
# Inspired by
# https://github.com/albumentations-team/albumentations/blob/master/albumentations/augmentations/functional.py#L1259
MAX_VALUES_BY_DTYPE_STR = {
    "uint8": 255,
    "uint16": 65535,
    "uint32": 4294967295,
    "float32": 1.0,
    "float64": 1.0,
}


In [ ]:
# hide
# I really would like to have an interface that easily combines all the channel information into a single class
from dataclasses import dataclass
from enum import IntEnum, Enum


class SpatialResolution(Enum):
    """
    Spatial resolution in square meters
    """

    TEN_METERS = 10
    TWENTY_METERS = 20
    SIXTY_METERS = 60


# No! The information should already be combined!
# There should be one place were the channel name is given
# and the resolution and default use-case is returned!


class Sentinel2_Channel(Enum):
    B01 = "B01"
    B02 = "B02"
    B03 = "B03"
    B04 = "B04"
    B05 = "B05"
    B06 = "B06"
    B07 = "B07"
    B08 = "B08"
    B8A = "B8A"
    B09 = "B09"
    B10 = "B10"
    B11 = "B11"
    B12 = "B12"


@dataclass
class BEN_Channel:
    channel: Sentinel2_Channel
    information: str
    # needs to be naturally ordered
    resolution: SpatialResolution


In [ ]:
# export
URL = "http://bigearth.net/downloads/BigEarthNet-v1.0.tar.gz"

# Stats from https://git.tu-berlin.de/rsim/bigearthnet-models-tf/-/blob/master/BigEarthNet.py
BAND_STATS = {
    "mean": {
        "B01": 340.76769064,
        "B02": 429.9430203,
        "B03": 614.21682446,
        "B04": 590.23569706,
        "B05": 950.68368468,
        "B06": 1792.46290469,
        "B07": 2075.46795189,
        "B08": 2218.94553375,
        "B8A": 2266.46036911,
        "B09": 2246.0605464,
        "B11": 1594.42694882,
        "B12": 1009.32729131,
    },
    "std": {
        "B01": 554.81258967,
        "B02": 572.41639287,
        "B03": 582.87945694,
        "B04": 675.88746967,
        "B05": 729.89827633,
        "B06": 1096.01480586,
        "B07": 1273.45393088,
        "B08": 1365.45589904,
        "B8A": 1356.13789355,
        "B09": 1302.3292881,
        "B11": 1079.19066363,
        "B12": 818.86747235,
    },
}

# FUTURE: Double check original values!
# Custom transformation to float32 values.
# Assumes that the original means and std values are encoded in uint16
BAND_STATS_FLOAT32 = {
    k: {
        band: band_val / MAX_VALUES_BY_DTYPE_STR["uint16"]
        for band, band_val in v.items()
    }
    for k, v in BAND_STATS.items()
}

BEN_CHANNELS = (
    "B01",
    "B02",
    "B03",
    "B04",
    "B05",
    "B06",
    "B07",
    "B08",
    "B8A",
    "B09",
    "B11",
    "B12",
)

BEN_10m_CHANNELS = (
    "B02",
    "B03",
    "B04",
    "B08",
)

BEN_20m_CHANNELS = (
    "B05",
    "B06",
    "B07",
    "B8A",
    "B11",
    "B12",
)

BEN_10m_20m_CHANNELS = natsort.natsorted(BEN_10m_CHANNELS + BEN_20m_CHANNELS)

BEN_30m_CHANNELS = (
    "B01",
    "B09",
)

# Correctly ordered
BEN_RGB_CHANNELS = (
    "B04",
    "B03",
    "B02",
)


In [ ]:
# hide
fc.test_eq(
    set(BEN_CHANNELS), set(BEN_10m_CHANNELS + BEN_20m_CHANNELS + BEN_30m_CHANNELS)
)
assert len(set(BEN_10m_CHANNELS) & set(BEN_20m_CHANNELS) & set(BEN_30m_CHANNELS)) == 0


In [ ]:
# export
COUNTRIES = (
    "Austria",
    "Belgium",
    "Finland",
    "Ireland",
    "Kosovo",
    "Lithuania",
    "Luxembourg",
    "Portugal",
    "Serbia",
    "Switzerland",
)

# NOTE: ISO_A2 because ISO_A3 does is NOT defined for Kosovo!
COUNTRIES_ISO_A2 = (
    "AT",
    "BE",
    "FI",
    "IE",
    "XK",
    "LT",
    "LU",
    "PT",
    "RS",
    "CH",
)


In [ ]:
# hide

assert len(COUNTRIES) == len(COUNTRIES_ISO_A2) == 10


In [ ]:
# hide
# Get the coordinates of the two points defining the bounding box:
# [p1, p2] where pX in Lat/Long format and p1 south-west and
# p2 north-east coordinates of box.
# Only keeping it in notebook, if I need it later for a quick visualization library

_BEN_BOUNDING_BOX = (
    (36.9569567020834, -9.00023345437725),
    (68.0216820004728, 31.598439091981),
)
assert len(_BEN_BOUNDING_BOX) == 2
assert len(_BEN_BOUNDING_BOX[0]) == len(_BEN_BOUNDING_BOX[1]) == 2


In [ ]:
# export

BEN_COMPLETE_SIZE = 590_326
BEN_SNOWY_PATCHES_COUNT = 61_707
BEN_CLOUDY_OR_SHADOWY_PATCHES_COUNT = 9_280
# this is before removing cloudy/snowy patches
BEN_NO_19_CLASS_TARGET_COUNT = 57
BEN_RECOMMENDED_SIZE = 519_284

In [ ]:
# hide

# There is one cloudy patch without a 19-class target:
# S2A_MSIL2A_20171208T093351_1_64
# And one snowy patch without a 19-class target:
# S2B_MSIL2A_20180417T102019_30_1

# there are no patches that are cloudy and snowy!

assert BEN_COMPLETE_SIZE == (
    BEN_RECOMMENDED_SIZE
    + BEN_SNOWY_PATCHES_COUNT
    + BEN_CLOUDY_OR_SHADOWY_PATCHES_COUNT
    + BEN_NO_19_CLASS_TARGET_COUNT
    - 2
)


In [ ]:
# export


def print_all_constants():
    """
    A function that shows all of the pre-defined constants of the library.
    """
    for k, v in globals().items():
        if not k.startswith("_") and k.upper() == k:
            rich.print(f"{k}:", v)


if __name__ == "__main__":
    print_all_constants()


OLD2NEW_LABELS_DICT:
{
    'Continuous urban fabric': 'Urban fabric',
    'Discontinuous urban fabric': 'Urban fabric',
    'Industrial or commercial units': 'Industrial or commercial units',
    'Non-irrigated arable land': 'Arable land',
    'Permanently irrigated land': 'Arable land',
    'Rice fields': 'Arable land',
    'Vineyards': 'Permanent crops',
    'Fruit trees and berry plantations': 'Permanent crops',
    'Olive groves': 'Permanent crops',
    'Annual crops associated with permanent crops': 'Permanent crops',
    'Pastures': 'Pastures',
    'Complex cultivation patterns': 'Complex cultivation patterns',
    'Land principally occupied by agriculture, with significant areas of natural vegetation':
'Land principally occupied by agriculture, with significant areas of natural vegetation',
    'Agro-forestry areas': 'Agro-forestry areas',
    'Broad-leaved forest': 'Broad-leaved forest',
    'Coniferous forest': 'Coniferous forest',
    'Mixed forest': 'Mixed forest',
    'Natural grassland': 'Natural grassland and sparsely vegetated areas',
    'Sparsely vegetated areas': 'Natural grassland and sparsely vegetated areas',
    'Moors and heathland': 'Moors, heathland and sclerophyllous vegetation',
    'Sclerophyllous vegetation': 'Moors, heathland and sclerophyllous vegetation',
    'Transitional woodland/shrub': 'Transitional woodland, shrub',
    'Beaches, dunes, sands': 'Beaches, dunes, sands',
    'Inland marshes': 'Inland wetlands',
    'Peatbogs': 'Inland wetlands',
    'Salt marshes': 'Coastal wetlands',
    'Salines': 'Coastal wetlands',
    'Water courses': 'Inland waters',
    'Water bodies': 'Inland waters',
    'Coastal lagoons': 'Marine waters',
    'Estuaries': 'Marine waters',
    'Sea and ocean': 'Marine waters',
    'Airports': None,
    'Bare rock': None,
    'Dump sites': None,
    'Port areas': None,
    'Road and rail networks and associated land': None,
    'Mineral extraction sites': None,
    'Construction sites': None,
    'Sport and leisure facilities': None,
    'Burnt areas': None,
    'Intertidal flats': None,
    'Green urban areas': None
}

OLD_LABELS:
[
    'Agro-forestry areas',
    'Airports',
    'Annual crops associated with permanent crops',
    'Bare rock',
    'Beaches, dunes, sands',
    'Broad-leaved forest',
    'Burnt areas',
    'Coastal lagoons',
    'Complex cultivation patterns',
    'Coniferous forest',
    'Construction sites',
    'Continuous urban fabric',
    'Discontinuous urban fabric',
    'Dump sites',
    'Estuaries',
    'Fruit trees and berry plantations',
    'Green urban areas',
    'Industrial or commercial units',
    'Inland marshes',
    'Intertidal flats',
    'Land principally occupied by agriculture, with significant areas of natural vegetation',
    'Mineral extraction sites',
    'Mixed forest',
    'Moors and heathland',
    'Natural grassland',
    'Non-irrigated arable land',
    'Olive groves',
    'Pastures',
    'Peatbogs',
    'Permanently irrigated land',
    'Port areas',
    'Rice fields',
    'Road and rail networks and associated land',
    'Salines',
    'Salt marshes',
    'Sclerophyllous vegetation',
    'Sea and ocean',
    'Sparsely vegetated areas',
    'Sport and leisure facilities',
    'Transitional woodland/shrub',
    'Vineyards',
    'Water bodies',
    'Water courses'
]

NEW_LABELS:
[
    'Agro-forestry areas',
    'Arable land',
    'Beaches, dunes, sands',
    'Broad-leaved forest',
    'Coastal wetlands',
    'Complex cultivation patterns',
    'Coniferous forest',
    'Industrial or commercial units',
    'Inland waters',
    'Inland wetlands',
    'Land principally occupied by agriculture, with significant areas of natural vegetation',
    'Marine waters',
    'Mixed forest',
    'Moors, heathland and sclerophyllous vegetation',
    'Natural grassland and sparsely vegetated areas',
    'Pastures',
    'Permanent crops',
    'Transitional woodland, shrub',
    'Urban fabric'
]

MAX_VALUES_BY_DTYPE_STR:
{'uint8': 255, 'uint16': 65535, 'uint32': 4294967295, 'float32': 1.0, 'float64': 1.0}

URL: http://bigearth.net/downloads/BigEarthNet-v1.0.tar.gz

...

BEN_SNOWY_PATCHES_COUNT: 61707

BEN_CLOUDY_OR_SHADOWY_PATCHES_COUNT: 9280

BEN_NO_19_CLASS_TARGET_COUNT: 57

BEN_COMPLETE_SIZE: 590326

BEN_RECOMMENDED_SIZE: 519284

In [ ]:
# hide
from nbdev.cli import nbdev_build_docs
from nbdev.export import notebook2script

notebook2script()
nbdev_build_docs()


Converted 01a_constants.ipynb.
Converted 01b_base.ipynb.
Converted 01c_gdf_builder.ipynb.
Converted 01d_subset_builder.ipynb.
Converted index.ipynb.
converting: /home/kai/git/bigearthnet_common/nbs/01a_constants.ipynb
converting /home/kai/git/bigearthnet_common/nbs/index.ipynb to README.md
